In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='YourGroq_API_Key', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("Latest developments in python")
print(response.content)

**Latest Developments in Python**

Python is a rapidly evolving language, with new features and improvements being added regularly. Here are some of the latest developments in Python:

### **Python 3.11**

Python 3.11 was released in October 2022, and it includes several new features and improvements, such as:

* **Exception Groups**: A new way to handle multiple exceptions at once, making it easier to write robust error handling code.
* **Self Type**: A new type hint that allows you to specify the type of the current instance, making it easier to write type-safe code.
* **Var, Const, and Final**: New type hints that allow you to specify the mutability of variables, making it easier to write readable and maintainable code.

### **Python 3.12**

Python 3.12 is currently in development, and it is expected to be released in October 2023. Some of the new features and improvements that are planned for Python 3.12 include:

* **Improved Error Messages**: Python 3.12 will include improved err

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-38714?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Data Engineer I, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsData Engineer I, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. Team
NIKE, Inc. does m

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: 'role', 'experience', 'skills','softskills' and 'description'.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Data Engineer I, ITC',
 'experience': '0-2 years',
 'skills': ['Big data tech stack knowledge',
  'PySpark',
  'SQL',
  'Databricks',
  'Snowflake',
  'Airflow'],
 'softskills': ['Excellent written and verbal communication skills',
  'Problem-solving skills'],
 'description': 'Building, enhancing, and troubleshooting complex data pipelines. Collaborating with product managers, engineers, analysts to build, enhance and troubleshoot data pipelines. Collaborate with senior, lead and principal engineers to define and implement quality standards across data pipelines. Contribute towards the design and architecture of data pipelines. Implement data quality and reliability measures across data pipelines.'}

In [8]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [15]:
links = collection.query(query_texts=['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [13]:
job = json_res
job['skills']


['Big data tech stack knowledge',
 'PySpark',
 'SQL',
 'Databricks',
 'Snowflake',
 'Airflow']

In [16]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [18]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are XYZ, a ML engineer in ABCDE. ABCDE is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ABCDE 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase ABCDE's portfolio: {link_list}
        Remember you are XYZ, ML engineer at ABCDE. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Engineering Solutions for Your Business

Dear Hiring Manager,

I came across the Data Engineer I, ITC role at your organization and was impressed by the opportunity to work on building, enhancing, and troubleshooting complex data pipelines. As a Machine Learning Engineer at ABCDE, I'd like to introduce you to our company's capabilities in delivering top-notch data engineering solutions.

At ABCDE, we specialize in empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in designing and implementing data pipelines using Big Data technologies, PySpark, SQL, Databricks, Snowflake, and Airflow.

We understand the importance of data quality and reliability measures across data pipelines. Our team is well-equipped to collaborate with your product managers, engineers, and analysts to build, enhance, and troubleshoot data pipelines that meet